# Lab 04

## Conrad Appel, Erik Gabrielsen, Danh Nguyen

### Preparation and Overview (30 points total)

[5 points] Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). Detail exactly what the classification task is and what parties would be interested in the results.

[10 points] (mostly the same processes as from lab one) Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

[15 points] Divide you data into training and testing data using an 80% training and 20% testing split. Use the cross validation modules that are part of scikit-learn. Argue for or against splitting your data using an 80/20 split. That is, why is the 80/20 split appropriate (or not) for your dataset?  

### Modeling (50 points total)

[20 points] Create a custom, one-versus-all logistic regression classifier using numpy and scipy to optimize. Use object oriented conventions identical to scikit-learn. You should start with the template used in the course. You should add the following functionality to the logistic regression classifier:
Ability to choose optimization technique when class is instantiated: either steepest descent, stochastic gradient descent, or Newton's method. Update the gradient calculation to include a customizable regularization term (either using no regularization, L1 regularization, L2 regularization, or both L1/L2 norm of the weights). Associate a cost with the regularization term, "C", that can be adjusted when the class is instantiated.  

[15 points] Train your classifier to achieve good generalization performance. That is, adjust the optimization technique and the value of the regularization term "C" to achieve the best performance on your test set. Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?

[15 points] Compare the performance of your "best" logistic regression optimization procedure to the procedure used in scikit-learn. Visualize the performance differences in terms of training time, training iterations, and memory usage while training. Discuss the results. 

### Deployment (10 points total)

Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

### Exceptional Work (10 points total)

You have free reign to provide additional analyses.
One idea: Make your implementation of logistic regression compatible with the GridSearchCV function that is part of scikit-learn.

In [18]:
## redoing large files
import pandas as pd
import numpy as np

%time df = pd.read_csv('~/Downloads/ss15pusb.csv') # read in the csv file


CPU times: user 51.5 s, sys: 14.8 s, total: 1min 6s
Wall time: 1min 14s


In [20]:
df.columns.values

array(['RT', 'SERIALNO', 'SPORDER', 'PUMA', 'ST', 'ADJINC', 'PWGTP',
       'AGEP', 'CIT', 'CITWP', 'COW', 'DDRS', 'DEAR', 'DEYE', 'DOUT',
       'DPHY', 'DRAT', 'DRATX', 'DREM', 'ENG', 'FER', 'GCL', 'GCM', 'GCR',
       'HINS1', 'HINS2', 'HINS3', 'HINS4', 'HINS5', 'HINS6', 'HINS7',
       'INTP', 'JWMNP', 'JWRIP', 'JWTR', 'LANX', 'MAR', 'MARHD', 'MARHM',
       'MARHT', 'MARHW', 'MARHYP', 'MIG', 'MIL', 'MLPA', 'MLPB', 'MLPCD',
       'MLPE', 'MLPFG', 'MLPH', 'MLPI', 'MLPJ', 'MLPK', 'NWAB', 'NWAV',
       'NWLA', 'NWLK', 'NWRE', 'OIP', 'PAP', 'RELP', 'RETP', 'SCH', 'SCHG',
       'SCHL', 'SEMP', 'SEX', 'SSIP', 'SSP', 'WAGP', 'WKHP', 'WKL', 'WKW',
       'WRK', 'YOEP', 'ANC', 'ANC1P', 'ANC2P', 'DECADE', 'DIS', 'DRIVESP',
       'ESP', 'ESR', 'FHICOVP', 'FOD1P', 'FOD2P', 'HICOV', 'HISP', 'INDP',
       'JWAP', 'JWDP', 'LANP', 'MIGPUMA', 'MIGSP', 'MSP', 'NAICSP',
       'NATIVITY', 'NOP', 'OC', 'OCCP', 'PAOC', 'PERNP', 'PINCP', 'POBP',
       'POVPIP', 'POWPUMA', 'POWSP', 'PRIVCOV', 'PUBC

In [29]:
df.drop_duplicates()

,RT,SERIALNO,SPORDER,PUMA,ST,ADJINC,PWGTP,AGEP,CIT,CITWP,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,76,1,1100,29,1001264,79,77,1,NaN,...,163,79,79,145,7,81,6,6,80,79
1,P,179,1,2800,29,1001264,70,30,1,NaN,...,129,69,69,73,16,71,16,70,74,125
2,P,326,1,1200,29,1001264,11,65,1,NaN,...,4,13,21,4,2,21,9,9,19,22
3,P,326,2,1200,29,1001264,12,69,1,NaN,...,3,13,21,5,2,20,11,11,16,23
4,P,360,1,600,29,1001264,177,24,1,NaN,...,260,67,186,44,168,152,196,70,53,47
5,P,360,2,600,29,1001264,148,25,1,NaN,...,277,52,132,36,134,134,138,48,48,36
6,P,378,1,200,29,1001264,67,20,1,NaN,...,18,20,74,63,59,106,69,139,120,37
7,P,378,2,200,29,1001264,61,19,1,NaN,...,18,18,89,77,38,62,43,105,71,21
8,P,425,1,1701,29,1001264,70,58,1,NaN,...,122,71,20,22,20,73,132,20,70,75
9,P,425,2,1701,29,1001264,101,53,1,NaN,...,187,102,27,29,25,103,197,31,106,111


In [72]:
df.head(10)
df.shape

new_df = df.filter(items=['AGEP','PINCP','QTRBIR','JWAP','JWDP','YOEP','WAOB','RAC1P'])
new_df.shape
new_df.to_csv('~/Downloads/ss15pusb2.csv')

In [73]:
new_df.shape

(1528516, 8)

In [74]:
# delete younger than 18
new_df = new_df[new_df.AGEP >= 18]

# find null columns
new_df.columns[new_df.isnull().any()].tolist()

['JWAP', 'JWDP', 'YOEP']

In [76]:
#new_df[new_df['JWAP'].isnull()]
# clean out JWAP and JWDP

df_grouped = new_df.groupby(by=['JWAP', 'JWDP'])
new_df = new_df.groupby(by=['JWAP', 'JWDP']).transform('median')
new_df.columns[new_df.isnull().any()].tolist()


['YOEP']

In [ ]:
## fill null values in YOEP (Year of entrance into US)


In [55]:
import numpy as np
import pandas as p
from scipy.optimize import fmin_bfgs
from scipy.special import expit

In [81]:
class BinaryClassifierBase:
    def __init__(self, eta, iterations=20, cost=0.001):
        self.eta = eta
        self.cost = cost
        self.iters = iterations
    
    def fit(self, x, y):
        self.w_ = np.zeros((x.shape[1],1))
        for _ in range(self.iters):
            gradient = self._get_gradient(x,y)
            self.w_ += gradient*self.eta
    
    def predict_proba(self,x):
        return 1/(1+np.exp(-(x @ self.w_)))
    
    def predict(self, x):
        return (self.predict_proba(x)>0.5)
    
    
class BinaryStochDescClassifier(BinaryClassifierBase):
    def _get_gradient(self, x, y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(x[idx]) # get y difference (now scalar)
        gradient = x[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.cost
        return gradient
    
    
class BinarySteepDescClassifier(BinaryClassifierBase):
    def _get_gradient(self, x, y):
        ydiff = y-self.predict_proba(x).ravel()
        gradient = np.mean(x * ydiff[:,np.newaxis], axis=0)
        return gradient.reshape(self.w_.shape)

    
class BinaryNewtonClassifier(BinaryClassifierBase):
    def fit(self, x, y):
        def obj_fn(w, x, y, c):
            g = expit(x @ w)
            return -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + c*sum(w**2)
        
        def obj_grad(w, x, y, c):
            g = expit(x @ w)
            ydiff = y-g
            gradient = np.mean(x * ydiff[:,np.newaxis], axis=0)
            gradient = gradient.reshape(w.shape)
            gradient[1:] += -2 * w[1:] * c
            return -gradient
        
        self.w_ = fmin_bfgs(obj_fn, 
                            np.zeros((x.shape[1], 1)), 
                            fprime=obj_grad, 
                            args=(x, y, self.cost), 
                            gtol=1e-03, 
                            maxiter=self.iters, 
                            disp=False).reshape((x.shape[1], 1))

        
class LogRegClassifier:
    def __init__(self, eta, iterations=20, optimize='steepdesc', cost=0.001):
        typesofoptimize = {
            'steepdesc': BinarySteepDescClassifier, 
            'stochdesc': BinaryStochDescClassifier, 
            'newton': BinaryNewtonClassifier
        }
        if optimize not in typesofoptimize.keys():
            raise ValueError('optimize must be one of: ' + ' '.join(typesofoptimize.keys()))
        
        self.eta = eta
        self.iters = iterations
        self.optimize = optimize
        self.classifier = typesofoptimize[optimize]
        self.cost = cost
        self.classifiers = [] # fill with binary classifiers during fit
    
    def _add_bias(self, x):
        return np.hstack((np.ones((x.shape[0],1)),x))
    
    def fit(self, x, y):
        Xb = self._add_bias(x)
        classes = np.unique(y)
        
        for cl in classes:
            cur_y = y==cl
            cur_classifier = self.classifier(self.eta, self.iters, cost=self.cost)
            cur_classifier.fit(x, cur_y)
            self.classifiers.append(cur_classifier)
    
    def predict(self, x):
        if not self.classifiers:
            raise RuntimeError('Classifier not fit!')
        
        probabilities = []
        for classifier in self.classifiers:
            probabilities.append(classifier.predict_proba(x))
        probabilities = np.hstack(probabilities)
        return np.argmax(probabilities,axis=1)

In [82]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
ds = load_iris()
regrs = {
    'newton': LogRegClassifier(.1, 500, optimize='newton'),
    'stoch': LogRegClassifier(.1, 500, optimize='stochdesc'), # Different results every time, random
    'steep': LogRegClassifier(.1, 500, optimize='steepdesc')
}
for regr in regrs.values():
    regr.fit(ds.data, ds.target)

for key, val in regrs.items():
    res = val.predict(ds.data)
    print(key + ': ' +str(accuracy_score(ds.target, res)))

steep: 0.96
stoch: 0.686666666667
newton: 0.966666666667
